In [2]:
#Reload modules without shutting notebook
%load_ext autoreload
%autoreload 2

import os
import glob
import numpy as np
from utils.get_configuration import get_configuration

# Get configuration information
cstudy = "skye" 
#cstudy = "cairngorms" 
#cstudy = "pdistrict" 

basefolder, s1ardfolder, datasets, outfolder, tmpfolder, ofiles, hdfile, pfile, verbose, graphics = get_configuration(cstudy)
if cstudy == "skye":
    aoi = os.path.join(datasets, "Skye_extent_OSGB36.geojson")
elif cstudy == "cairngorms":
    aoi = os.path.join(datasets, "Cairngorms_extent_OSGB36.geojson")    
else:
    aoi = os.path.join(datasets, "PDistrict_extent_OSGB36.geojson")
fstart = os.path.basename(aoi).split("_")[0]
subarray = None
print("Check: ",ofiles[0])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loaded configuration: {'sharedfolder': 'my_shared_data_folder'}.
Check:  /home/slavender/my_shared_data_folder/skye/S1B_20190201_30_asc_175839_175904_VVVH_G0_GB_OSGB_RTCK_SpkRL.tif


In [42]:
# Choose Burn Polygon
from ipywidgets import Dropdown

searchstr = fstart + "_burn_extent_*.geojson"
polygons = glob.glob(os.path.join(datasets, searchstr))
if len(polygons) == 0:
    print("No polygons exist for {}".format(searchstr))
poly_list = []
for poly in polygons:
    fpoly = os.path.splitext(os.path.basename(poly))[0]
    poly_list.append(fpoly.split("_")[3])

# Convert string list to int list, sort then convert back
poly_int = [int(i) for i in poly_list]
poly_int.sort()
poly_list = [str(i).zfill(3) for i in poly_int]
del poly_int

b_widget = Dropdown(options = poly_list, description = "Polygon:", value = poly_list[0])
def change_x(*args):
    print("Set to {}".format(b_widget.value))
    
b_widget.observe(change_x, 'value')
print("Choose {} burn area".format(cstudy))
display(b_widget)

Choose skye burn area


Dropdown(description='Polygon:', options=('006', '009', '016', '026', '054', '110', '143', '153', '156', '160'…

In [57]:
# Set polygon from drop-down above
endstr = ".geojson"
burn = "Skye_burn_extent_" + b_widget.value
baseline = "Skye_baseline_extent_" + b_widget.value
if os.path.exists(os.path.join(datasets, burn + endstr)):
    print("Chosen polygon: {}".format(burn))
    if os.path.exists(os.path.join(datasets, baseline + endstr)):
        print("Baseline non-burn polygon available")
        bdata = True
    else:
        print("No baseline data available")
        bdata = False
else:
    print("ERROR: chosen polygon {} not found".format(burn))

Chosen polygon: Skye_burn_extent_326
Baseline non-burn polygon available


In [58]:
# Load ERA5Land data
graphics = False # do not display graphs
polygon = burn
%run ./Pull-ERA5.ipynb $polygon $verbose $graphics
if bdata:  # run for baseline polygon:
    polygon = baseline
    %run ./Pull-ERA5.ipynb $polygon $verbose $graphics

Input dataset from: /home/slavender/my_shared_data_folder
Already downloaded: /home/slavender/temp/Skye_burn_extent_326-download.nc 
Loading: odict_keys(['longitude', 'latitude', 'time', 'u10', 'v10', 'lai_hv', 'lai_lv', 'snowc', 'sp', 'sro', 'ssrd', 'tp'])  
Extracted data shape:  (365, 9)
Input dataset from: /home/slavender/my_shared_data_folder
Already downloaded: /home/slavender/temp/Skye_baseline_extent_326-download.nc 
Loading: odict_keys(['longitude', 'latitude', 'time', 'u10', 'v10', 'lai_hv', 'lai_lv', 'snowc', 'sp', 'sro', 'ssrd', 'tp'])  
Extracted data shape:  (365, 9)


In [59]:
# Load Sentinel-1 ARD backscatter data for burn extent polygon
subarray = None # So runs again - ToDo to update so if run for same polygon does not reload
polygon = burn
%run ./Analyse-Sentinel1.ipynb $polygon $verbose
burn_subarray = subarray
burn_df = df
del df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Input dataset from: /home/slavender/my_shared_data_folder
Loading layer 44 of 44
 Extracting only polygon area...


interactive(children=(Dropdown(description='polarisation', options=('VV', 'VH'), value='VV'), IntSlider(value=…

interactive(children=(Button(description='Save', style=ButtonStyle()), Output()), _dom_classes=('widget-intera…

In [60]:
# Load Sentinel-1 ARD backscatter data for baseline polygon
subarray = None # So runs again - ToDo to update so if run for same polygon does not reload
if bdata:
    polygon = baseline
    %run ./Analyse-Sentinel1.ipynb $polygon $verbose
    baseline_subarray = subarray
    baseline_df = df
    del df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Input dataset from: /home/slavender/my_shared_data_folder
Loading layer 44 of 44
 Extracting only polygon area...


interactive(children=(Dropdown(description='polarisation', options=('VV', 'VH'), value='VV'), IntSlider(value=…

interactive(children=(Button(description='Save', style=ButtonStyle()), Output()), _dom_classes=('widget-intera…

In [61]:
# Check ERA5 and Sentinel-1 ARD data is loaded
print("Sentinel-1 ARD backscatter files in array of shape {} with {} files in dataframe".format(burn_subarray.shape,len(burn_df)))
if bdata:
    print("Baseline ARD in array of shape {} with {} files in dataframe".format(baseline_subarray.shape,len(baseline_df)))
print("ERA5 data: ", dt_num[0:5], mean_parameters.shape)

Sentinel-1 ARD backscatter files in array of shape (37, 2, 124, 132) with 37 files in dataframe
Baseline ARD in array of shape (37, 2, 71, 106) with 37 files in dataframe
ERA5 data:  ['20190101', '20190102', '20190103', '20190104', '20190105'] (365, 9)


In [62]:
# Choose filtering option
filter_list = []
headers = list(burn_df.head(0))
print("Headers: {}".format(headers))
for col in range(len(headers)):
    if col == 0:
        filter_list.append('All')
    elif headers[col] == 'etime' or headers[col] == 'polarisation':
        continue
    else:
        filter_list.append(headers[col])

# Sort alphabetically
filter_list.sort()
        
d_widget = Dropdown(options = filter_list, description = "Options:", value = filter_list[0])
def change_x(*args):
    print("Set to {}".format(d_widget.value))
    
d_widget.observe(change_x, 'value')
print("Choose filtering option")
display(d_widget)

Headers: ['layer', 'sensor', 'date', 'stime', 'etime', 'polarisation', 'rorbit', 'direction']
Choose filtering option


Dropdown(description='Options:', options=('All', 'date', 'direction', 'rorbit', 'sensor', 'stime'), value='All…

In [81]:
# Undertake sub-filtering based on option chosen above
if d_widget.value == 'All':
    print("Using all the data")
    options = ['All']
else:
    options = burn_df[d_widget.value].unique()
    #print(options)

# Sort alphabetically
options.sort()
        
sub_widget = Dropdown(options = options, description = "Sub Options:", value = options[0])
def change_x(*args):
    print("Set to {}".format(sub_widget.value))
    
sub_widget.observe(change_x, 'value')
if d_widget.value != 'All':
    print("Choose sub-filtering option")
    display(sub_widget)


Choose sub-filtering option


Dropdown(description='Sub Options:', options=('asc', 'desc'), value='asc')

In [82]:
# Filter the S1ARD data
from utils.subset_s1ard import subset_s1ard

filtered_burndf, filtered_burnarray, filtered_dates_burn = subset_s1ard(burn_df, burn_subarray, d_widget.value, sub_widget.value, verb = verbose)
if bdata:
    filtered_basedf, filtered_basearray, filtered_dates_base = subset_s1ard(baseline_df, baseline_subarray, d_widget.value, sub_widget.value, verb = verbose)


Subsetting to direction = asc
Original (37, 2, 124, 132) subsetted to 18 layers
Subsetting to direction = asc
Original (37, 2, 71, 106) subsetted to 18 layers


In [83]:
# Load CORINE land cover for polygon burn area subset
from utils.load_corine import subset_corine
cpolygons, legend_df = subset_corine(xymin[0], xymin[1], xymax[0], xymax[1], pixelWidth, pixelHeight, datasets, verb = verbose)

<open Collection '/home/slavender/my_shared_data_folder/datasets/u2018_clc2018_v2020_20u1_geoPackage/DATA/U2018_CLC2018_V2020_20u1.gpkg:U2018_CLC2018_V2020_20u1', mode 'r' at 0x7fac6bbb0a20>
Extracted 2 polygons


In [84]:
from utils.extract_s1ard import extract_s1ard
from utils.calc_era_api import calc_era_api

# Extract subsets of burn area using CORINE polygons
mean_s1burn, std_s1burn, pnames, features = extract_s1ard(cpolygons, filtered_burnarray, verb = verbose)
if bdata:
    mean_s1base, std_s1base, pnames, features = extract_s1ard(cpolygons, filtered_basearray, verb = verbose)
    
# Extract ERA5 data for matching dates and calculate Antecedent Precipitation Index
if d_widget.value == 'All':
    filtered_dates = burn_df['date'].values
else:
    filtered_dates = burn_df['date'].values[filtered_dates_burn]
mean_edata, std_edata = calc_era_api(dt_num, mean_parameters, filtered_burnarray, filtered_dates, verb = True)
filtered_edata = filtered_dates
eraname = 'Antecedent Precipitation Index'

# Print initial arrays
print("{} S1 dates: {} ...".format(len(list_of_dates),list_of_dates[0:5]))
print("S1ARD array {}".format(mean_s1burn.shape))
for fnum in range(len(features)):
    print("   Burn {} max VV {:.3f} max VH {:.3f} VHrVV {:.3f} RFDI {:.3f} NVHI {:.3f} NVVI {:.3f}"
        .format(features[fnum],np.nanmax(mean_s1burn[fnum,0,:]),np.nanmax(mean_s1burn[fnum,1,:]),np.nanmax(mean_s1burn[fnum,2,:]),
        np.nanmax(mean_s1burn[fnum,3,:]),np.nanmax(mean_s1burn[fnum,4,:]),np.nanmax(mean_s1burn[fnum,5,:])))
    if bdata:
        print("  Baseline {} max VV {:.3f} max VH {:.3f} VHrVV {:.3f} RFDI {:.3f} NVHI {:.3f} NVVI {:.3f}"
            .format(features[fnum],np.nanmax(mean_s1base[fnum,0,:]),np.nanmax(mean_s1base[fnum,1,:]),np.nanmax(mean_s1base[fnum,2,:]),
            np.nanmax(mean_s1base[fnum,3,:]),np.nanmax(mean_s1base[fnum,4,:]),np.nanmax(mean_s1base[fnum,5,:])))
print("ERA5 Precip: {} values max {:.3f}".format(len(mean_edata), np.nanmax(mean_edata)))


37 S1 dates: ['20190202' '20190205' '20190207' '20190301' '20190302'] ...
S1ARD array (2, 6, 18)
   Burn Moors and heathland max VV nan max VH nan VHrVV nan RFDI nan NVHI nan NVVI nan
  Baseline Moors and heathland max VV -10.753 max VH -18.408 VHrVV 2.003 RFDI -0.160 NVHI 0.667 NVVI 0.420
   Burn Peat bogs max VV -8.246 max VH -15.993 VHrVV 2.195 RFDI -0.247 NVHI 0.681 NVVI 0.376
  Baseline Peat bogs max VV -10.486 max VH -17.030 VHrVV 1.789 RFDI -0.194 NVHI 0.638 NVVI 0.403
ERA5 Precip: 18 values max 0.035


In [85]:
# Create plots for ERA5 data in comparison to Sentinel-1 ARD dates for each CORINE polygon
plt.set_loglevel("critical")

if d_widget.value != "All":
    print("Filtering: {} using {}".format(d_widget.value, sub_widget.value))
    
if bdata:
    outbase = os.path.join(outfolder,"plot-s1ard-era-baseline.png")
    if d_widget.value != "All":
        filtered_dates = baseline_df['date'].values[filtered_dates_base]
    else:
        filtered_dates = baseline_df['date'].values
    #print(filtered_edata.shape, mean_edata.shape, std_edata.shape)
    #print(filtered_dates.shape, mean_s1base.shape, std_s1base.shape)
    plotd.plot_lines(filtered_dates, mean_s1base, std_s1base, filtered_edata, mean_edata, std_edata, features, pnames, eraname, outbase, graphics = False)
    print("Generated: {}".format(outbase))

    outburn = os.path.join(outfolder,"plot-s1ard-era-burn.png")
    if d_widget.value != "All":
        filtered_dates = burn_df['date'].values[filtered_dates_burn]
    else:
        filtered_dates = burn_df['date'].values
    #print(filtered_edata.shape, mean_edata.shape, std_edata.shape)
    #print(filtered_dates.shape, mean_s1burn.shape, std_s1burn.shape)
    plotd.plot_lines(filtered_dates, mean_s1burn, std_s1burn, filtered_edata, mean_edata, std_edata, features, pnames, eraname, outbase, graphics = False)
    print("Generated: {}".format(outburn))
    
else:    
    outfile = os.path.join(outfolder,"plot-s1ard-era-burn.png")
    #print(len(filtered_dates), mean_s1burn.shape, std_s1burn.shape, mean_edata.shape, std_edata.shape, len(features), len(pnames))
    plotd.plot_lines(filtered_dates, mean_s1burn, std_s1burn, filtered_edata, mean_edata, std_edata, features, pnames, eraname, outfile, graphics = False)
    print("Generated: {}".format(outburn))


Filtering: direction using asc
Running for 2 features with 6 products
Generated: /home/slavender/output_folder/plot-s1ard-era-baseline.png
Running for 2 features with 6 products
Generated: /home/slavender/output_folder/plot-s1ard-era-burn.png


In [86]:
# Display plots side by side if have both baseline and burn data extracted
import ipywidgets as widgets

# read plot images
if bdata:
    img1 = open(outbase, 'rb').read()
    img2 = open(outburn, 'rb').read()

    # Text widgets
    t1 = widgets.Text(value='Baseline polygon')
    if d_widget.value == 'All':
        t2 = widgets.Text(value=' Plotting all available data for ' + b_widget.value)
    else:
        t2 = widgets.Text(value='Plotting ' + b_widget.value + ' for ' + d_widget.value + ' data filtered by ' + sub_widget.value)
    t3 = widgets.Text(value='Burn polygon')
    ## Create image widgets.
    i1 = widgets.Image(value=img1, width='50%')
    i2 = widgets.Image(value=img2, width='50%')
    ## Side by side thanks to HBox widget
    titles = widgets.HBox([t1, t2, t3], width='100%')
    plots = widgets.HBox([i1, i2], width='100%')
    ui = widgets.VBox([titles,plots])
    ## Finally, show.
    display(ui)

else:
    img2 = open(outburn, 'rb').read()

    # display images
    display(widgets.Image(value=img2))
    
